In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [9]:
ner = pd.read_csv('ner_dataset.csv',encoding='latin1')
ner.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O


In [0]:
ner['Sentence #'] = ner['Sentence #'].ffill()

In [0]:
X = ner.drop('POS',axis=1)

In [0]:
words = set(X.Word.values)

In [39]:
len(words)

35178

In [0]:
tags = set(X.Tag)

In [0]:
X_sent = X.groupby('Sentence #').apply(lambda s: [(w,t) for w,t in zip(s['Word'].values,s['Tag'].values)])

In [0]:
from keras.preprocessing.text import Tokenizer


In [0]:
max_len = 50

In [46]:
from keras.preprocessing.sequence import pad_sequences
tags2index = {t:i for i,t in enumerate(tags)}
y = [[tags2index[w[1]] for w in s] for s in X_sent]
y = pad_sequences(maxlen=max_len, sequences=y, padding="post", value=tags2index["O"])
y[15]

array([5, 0, 0, 1, 8, 8, 8, 0, 0, 0, 0, 0, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0], dtype=int32)

In [51]:
tags2index

{'B-art': 7,
 'B-eve': 2,
 'B-geo': 16,
 'B-gpe': 5,
 'B-nat': 15,
 'B-org': 12,
 'B-per': 1,
 'B-tim': 9,
 'I-art': 10,
 'I-eve': 11,
 'I-geo': 6,
 'I-gpe': 3,
 'I-nat': 4,
 'I-org': 14,
 'I-per': 8,
 'I-tim': 13,
 'O': 0}

In [0]:
#tok = Tokenizer(35178,filters='',lower=True)
X = [[str(w[0]) for w in s] for s in X_sent]

with open('tok.txt') as json_file:
    data = json.load(json_file)
from keras.preprocessing.text import tokenizer_from_json
tok = tokenizer_from_json(data)
len(tok.word_index)
tok.fit_on_texts(X)
X_seq = tok.texts_to_sequences(X)
X_padded = pad_sequences(maxlen=max_len,sequences=X_seq,padding="post")
X_padded.shape,y.shape

In [57]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X_padded,y,test_size=0.20,random_state = 77)
X_train,X_val,y_train,y_val = train_test_split(X_train,y_train,test_size = 0.15,random_state = 77)
X_train.shape,y_train.shape,X_val.shape,y_val.shape,X_test.shape,y_test.shape

((32611, 50), (32611, 50), (5756, 50), (5756, 50), (9592, 50), (9592, 50))

In [0]:
idx2tag = {i: w for w, i in tags2index.items()}

def pred2label(pred):
  out = [] 
  for pred_i in test_pred:
    out_i = []
    for p in pred_i:
      p_i = np.argmax(p)
      out_i.append(idx2tag[p_i])
    out.append(out_i)
  return out

def test2label(pred):
    out = []
    for pred_i in pred:
        out_i = []
        for p in pred_i:
            out_i.append(idx2tag[p])
        out.append(out_i)
    return out


In [0]:
from keras.models import Model, Input
from keras.layers.merge import add
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional, Lambda,Flatten

In [0]:
!pip install seqeval

In [0]:
from seqeval.metrics import precision_score, recall_score, f1_score, classification_report
#test_pred = model.predict(np.array(X_test), verbose=1)


In [60]:
!wget http://nlp.stanford.edu/data/glove.6B.zip


--2020-05-19 04:37:16--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2020-05-19 04:37:16--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2020-05-19 04:37:16--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [0]:
%%capture
!unzip glove.6B.zip

In [62]:
word_index = tok.word_index
print('Found %s unique tokens.' % len(word_index))

Found 31817 unique tokens.


In [63]:
embeddings_index = {}
import os
f = open(os.path.join('/content/', 'glove.6B.300d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [0]:
EMBEDDING_DIM = 300
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [0]:
from keras.layers import Embedding

embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=50,
                            trainable=False)

In [67]:
main_input = Input(shape=(50,),name = 'main_input')

emb = embedding_layer(main_input)
#emb = Embedding(output_dim=512,input_length=50,input_dim = 35178)(main_input)
l1 = Bidirectional(LSTM(512,return_sequences=True,recurrent_dropout=0.2,dropout=0.2))(emb)
l2 = Bidirectional(LSTM(512,return_sequences=True,recurrent_dropout=0.2,dropout=0.2))(l1)
#fl = Flatten()(l2)
x = add([l1,l2])
out = TimeDistributed(Dense(len(tags),activation="softmax"))(x)
model = Model(main_input,out)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
main_input (InputLayer)         (None, 50)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 50, 300)      9545400     main_input[0][0]                 
__________________________________________________________________________________________________
bidirectional_3 (Bidirectional) (None, 50, 1024)     3330048     embedding_1[1][0]                
__________________________________________________________________________________________________
bidirectional_4 (Bidirectional) (None, 50, 1024)     6295552     bidirectional_3[0][0]            
____________________________________________________________________________________________

In [0]:
from keras import optimizers as op

In [0]:
y_train_t = y_train.reshape(y_train.shape[0],y_train.shape[1],1)
y_val_t = y_val.reshape(y_val.shape[0],y_val.shape[1],1)

In [70]:
model.compile(loss='sparse_categorical_crossentropy',optimizer=op.Adam(lr=1e-3),metrics=['acc'])
model.fit(np.array(X_train),y_train_t,batch_size=32,epochs=3,
          validation_data=(np.array(X_val),y_val_t),)
model.save('bi_dir10_1.h5')

Train on 32611 samples, validate on 5756 samples
Epoch 1/3
32611/32611 [==============================] - 4813s 148ms/step - loss: 0.1010 - acc: 0.9727 - val_loss: 0.0617 - val_acc: 0.9812
Epoch 2/3
32611/32611 [==============================] - 4884s 150ms/step - loss: 0.0573 - acc: 0.9822 - val_loss: 0.0549 - val_acc: 0.9829
Epoch 3/3
32611/32611 [==============================] - 4896s 150ms/step - loss: 0.0485 - acc: 0.9844 - val_loss: 0.0505 - val_acc: 0.9839


In [71]:
test_pred = model.predict(X_test,verbose=1)

9592/9592 [==============================] - 247s 26ms/step


In [75]:
y_pt = pred2label(test_pred)
y_tt = test2label(y_test)
print(classification_report(y_tt, y_pt))

           precision    recall  f1-score   support

      tim       0.85      0.85      0.85      4107
      gpe       0.96      0.93      0.95      3173
      per       0.72      0.77      0.74      3309
      geo       0.84      0.87      0.86      7624
      org       0.58      0.60      0.59      3958
      nat       0.38      0.41      0.40        41
      art       0.14      0.04      0.06        82
      eve       0.62      0.31      0.41        58

micro avg       0.79      0.81      0.80     22352
macro avg       0.79      0.81      0.80     22352



In [80]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
text = 'Kanimozhi went to Mumbai'
def predict_on_text(text):
  words = word_tokenize(text)

  t_seq = []
  for i in tok.texts_to_sequences(words):
      try:
          t_seq.append(i[0])
      except:
          t_seq.append(0)
  t_seq_padded = np.array(pad_sequences(maxlen=max_len,sequences = [t_seq],padding="post")).reshape(1,50)
  pred = model.predict(t_seq_padded)
  p = np.argmax(pred, axis=-1)[0]
  print("{:15} : {}".format("Word", "Pred"))
  print("="*30)

  for j in range(len(words)):
      print("{:15} :  {}".format(words[j],idx2tag[p[j]]))

In [0]:
def print_seq(X,y,i):
  X = X[i]
  y = y[i]
  p = model.predict(np.array(X.reshape(1,-1)))
  p = np.argmax(p, axis=-1)[0]
  print("{:20} {:5}  : \t{}".format("Word", "Pred", "True"))
  print("="*40)
  words = tok.sequences_to_texts(X.reshape(1,50))[0].split()
 
  for j in range(len(words)):
    
    print("{:20} {:5}  : \t{}".format(words[j],idx2tag[p[j]],idx2tag[y[j]]))
i = 139
print_seq(X_train,y_train,i)

##Prediction

In [123]:
predict_on_text('Karthick went to Mumbai')

Word            : Pred
Karthick        :  O
went            :  O
to              :  O
Mumbai          :  B-geo


In [122]:
predict_on_text("Kanimozhi went to Mumbai from Chennai Railway Station yesterday night.")

Word            : Pred
Kanimozhi       :  O
went            :  O
to              :  O
Mumbai          :  B-geo
from            :  O
Chennai         :  B-geo
Railway         :  O
Station         :  O
yesterday       :  B-tim
night           :  I-tim
.               :  O


In [124]:
predict_on_text('India won in the Commonwealth Games')

Word            : Pred
India           :  B-geo
won             :  O
in              :  O
the             :  O
Commonwealth    :  B-org
Games           :  O


In [125]:
predict_on_text("Aakash is working for Infosys Ltd.")

Word            : Pred
Aakash          :  B-org
is              :  O
working         :  O
for             :  O
Infosys         :  O
Ltd             :  O
.               :  O


In [126]:
predict_on_text("Kamal went to Ibaco yesterday to eat ice cream")

Word            : Pred
Kamal           :  B-per
went            :  O
to              :  O
Ibaco           :  O
yesterday       :  B-tim
to              :  O
eat             :  O
ice             :  O
cream           :  O
